In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_dbconn, utc
import numpy as np

In [5]:
STORM_NUMBER = 8
TITLE = "27 December 2020"
SUBTITLE = "8 AM 28 December 2020"
sts = utc(2020, 12, 27, 6)
ets = utc(2020, 12, 28, 23, 0)
# Get available data
ctx = {'coop': 'yes', 'csector': 'IA', 't': 'state', 'sz': 30, 'z': 'yes', 'f': 'linear'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx)
# add zeros and QC
df = add_zeros(df, ctx)

In [6]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s, "
                  "IEM 2020-2021 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df_useme_plot)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'],
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=0)
    return mp

In [28]:
cull = ['SWEI4', 'CIYI4', 'IA-ID-4', 'AKYI4', 'MSHI4', 'IWAI4', 173, 'ICYI4', 'IA-IA-13', 174, 'WLBI4', 'MKTI4',
       'CRPI4', 'AMOI4', 'EKRI4', 114]
if cull:
    df.loc[df['nwsli'].isin(cull), USEME] = False
# df.loc[df[df["state"] == "MN"].index, USEME] = False
hardcode = [
]
for nwsli, val in hardcode:
    df.loc[df['nwsli'] == nwsli, 'val'] = val

ctx['csector'] = "IA"
mp = workflow(ctx, df, isfinal=True, lower=1, upper=2)
res = mp.postprocess(filename='201228.png')
mp.close()

    state  wfo     val        lon        lat                             geo  \
102    WI  MKX  1.9000 -89.520000  43.090000  POINT (848222.697 -157824.492)   
103    WI  MKX  1.9000 -89.410000  43.060000  POINT (857488.128 -159989.309)   
106    WI  ARX  1.8000 -90.670000  43.510000  POINT (750430.237 -122657.695)   
108    IA  DVN  1.8000 -90.880000  42.580000  POINT (744957.642 -227503.282)   
112    WI  ARX  1.8000 -91.160000  43.720000  POINT (708718.151 -103806.451)   
116    WI  MKX  1.7000 -89.490000  43.050000  POINT (851195.634 -161932.573)   
119    IA  DVN  1.6000 -90.920000  41.860000  POINT (750341.807 -307556.418)   
120    WI  MKX  1.5000 -90.130000  43.580000  POINT (792676.532 -109795.521)   
124    IA  ARX  1.5000 -91.830000  42.880000  POINT (664437.626 -202476.225)   
125    IA  DVN  1.5000 -90.730000  42.490000  POINT (758252.058 -236084.877)   
133    WI  ARX  1.5000 -91.140000  43.040000  POINT (718398.000 -178913.375)   
134    WI  ARX  1.5000 -91.140000  43.06